# Constrained optimization: Penalty Method
## Giacomo Bacchetta, bacchetta.1840949@studenti.uniroma1.it
## Edoardo Cesaroni, cesaroni.1841742@studenti.uniroma1.it 
## Fabio Ciccarelli, ciccarelli.1835348@studenti.uniroma1.it 

## Support's functions

We import the python libraries useful for the implementation of the method. These are:
- Autograd.numpy
- Autograd

In [1]:
import autograd.numpy as np
from autograd import grad

## Unconstrained Optimization

We choose the Truncated Newton Method as the unconstrained optimization one into the Penalty Method.
There is another repository about this unconstrained optimization method. It is availble in the link: 

In [2]:
def armijo(f, x_k, d):
    alpha = 1
    gamma = 1e-3

    while True:
        if f(x_k + alpha*d) > f(x_k) + alpha*gamma*(grad(f)(x_k) @ d):
            alpha = 0.5*alpha
        else:
            return alpha

In [3]:
def mxv(v, f, x):
    eta = 1e-6
    x_succ = x + eta*v
    return (grad(f)(x_succ) - grad(f)(x))/eta

def gradfi(d, f, x):
    return mxv(d,f,x) + grad(f)(x)

In [4]:
def dt(f, x, k):

    epsilon_1 = 0.5
    epsilon_2 = 0.5
    p = 0

#     s = -(gradfi(p, f, x))
    s = -grad(f)(x)
    
    if (s @ mxv(s, f, x)) < (epsilon_1 * (np.linalg.norm(s))**2):
        d = -(grad(f)(x))
        return d
    
    while True:
        
        if (s @ mxv(s, f, x)) <= 1e-6:              #default = 1e-9
            return -grad(f)(x)
        
        alfa = -((gradfi(p, f, x) @ s) / (s @ mxv(s, f, x)))
        p = p + alfa * s

        if np.linalg.norm(gradfi(p, f, x)) <= (1/(k+1))*epsilon_2*(np.linalg.norm(grad(f)(x))):
            d = p
            return d
        
        else:
            beta = (gradfi(p, f, x) @ mxv(s, f, x)) / (s @ mxv(s, f, x))
            s = -(gradfi(p, f, x)) + beta * s

            if (s @ mxv(s, f, x)) < (epsilon_1 * (np.linalg.norm(s))**2):
                d = p
                return d

In [5]:
def nt(f, x, eps = 1e-5):  
    
    k = 0
    
    if np.linalg.norm(grad(f)(x)) < eps:
        return x
    
    while True:

        d = dt(f, x, k)
        a = armijo(f, x, d)
        x = x + a * d
        
        if np.linalg.norm(grad(f)(x)) <= eps:
            return x

        k = k + 1

## KKT Conditions

In [6]:
def verify_KKT(f, g, h, x, lam, mu):
    epsilon = 1e-3
    
    for i in range(len(g)):
        if g[i](x) > epsilon or lam[i]*g[i](x) > epsilon or lam[i]*g[i](x) < -epsilon:
            return False
        
    for j in range(len(h)):
        if h[j](x) > epsilon or h[j](x) < -epsilon:
            return False

    if np.linalg.norm(grad(f)(x) + sum(grad(g[i])(x)*lam[i] for i in range(len(g))) + sum(grad(h[j])(x)*mu[j] for j in range(len(h)))) > epsilon:
        return False

    return True

## Main

Before moving to the core of the algorithm, we defined the P function that replicates the quadratic penalty function.

In [7]:
def P(x):
    return f(x)+ (1/epsilon_p)*sum(max(0, g[i](x))**2 for i in range(len(g)))+ (1/epsilon_p)*sum(h[j](x)**2 for j in range(len(h)))

In [8]:
g = []
h = []

def penalty_method(f, g, h, x):
    global epsilon_p

    theta1 = 0.5
    theta2 = 0.5
    theta3 = 0.5
    delta = 10
    epsilon_prev = 0.1                  #alternative value = 1
    epsilon_current = 0.1
    epsilon_p = epsilon_current
    ite = 0
    
    
    while True:
        
        lam = []
        mu = []

        for i in range(len(g)):
            lam.append((max(0, g[i](x)))*(2/epsilon_prev))

        for j in range(len(h)):
            mu.append((h[j](x))*(2/epsilon_prev))

        verifica_KKT = verify_KKT(f, g, h, x, lam, mu)
        if verify_KKT(f, g, h, x, lam, mu) == True:
            print('*'*70)
            print('PM: STOP! KKT condition is verified.')
            print('PM: The optimum point is:',x, 'with lambda=', lam, 'and mu=', mu)
            print('PM: The optimum value of the bbjective function is:', f(x))
            return x

        epsilon_p = epsilon_current


        x_succ = nt(P, x, delta)
    

        if sum(max(0, g[i](x_succ))**2 for i in range(len(g))) + sum((h[j](x_succ))**2 for j in range(len(h))) <= theta1*(sum(max(0, g[i](x))**2 for i in range(len(g))) + sum((h[j](x))**2 for j in range(len(h)))):
            epsilon_prev = epsilon_current
            epsilon_current = epsilon_current
            
        elif epsilon_current >= 1e-6: 
            epsilon_prev = epsilon_current
            epsilon_current = theta2 * epsilon_current
            
        elif epsilon_current < 1e-6:
            print('*'*70)
            print('PM: STOP! Epsilon too little, the best point is:', x_succ)
            print('PM: The best value of the objective function is:', f(x_succ))
            return x_succ

        if delta > 1e-4:
            delta = delta * theta3
        x = x_succ
        
        ite = ite + 1

## Implementation test

### PROBLEMA 1.1

In [9]:
# def f(x):
#     return (x[0]-5.)**2 + x[1]**2 - 25

# def g1(x):
#     return -(-x[0]**2 + x[1])
# g.append(g1)

# x0 = np.array([4.9, 0.1])


# penalty_method(f, g, h, x0)

## PROBLEMA 1.2

In [10]:
# def f(x):
#     f = 0.5*x[0]**2+x[1]**2-x[0]*x[1]-7*x[0]-7*x[1]
#     return f

# def g1(x):
#     return -25 + 4*x[0]**2 + x[1]**2
# g.append(g1)

# x0 = np.array([0., 0.])


# penalty_method(f, g, h, x0)

## PROBLEMA 1.3

In [11]:
# def f(x):
#     return (x[0]-2)**2 + x[1]**2

# def g1(x):
#     return -((1-x[0])**3) + x[1]
# g.append(g1)

# def g2(x):
#     return -x[0]
# g.append(g2)

# def g3(x):
#     return -x[1]
# g.append(g3)

# x0 = np.array([-2., -2.])


# penalty_method(f, g, h, x0)

## PROBLEMA 3.18

In [12]:
# def f(x):
#     return (x[0]-1)**2+(x[1]-x[2])**2+(x[3]-x[4])**2

# def h1(x):
#     return (x[0]+x[1]+[2]+x[3]+[4]-5)
# h.append(h1)

# def h2(x):
#     return (x[2]-2*(x[3]+x[4])+3)
# h.append(h2)

# x0 = np.array([3.0,5.0,-3.0,2.0,-2.0])


# penalty_method(f, g, h, x0)

## PROBLEMA 3.23

In [13]:
# def f(x):
#     return (x[0]-x[1])**2+(x[1]+x[2]-2)**2+(x[3]-1)**2+(x[4]-1)**2

# def h1(x):
#     return (x[0]+3*x[1])
# h.append(h1)

# def h2(x):
#     return (x[2]+x[3]-2*x[4])
# h.append(h2)

# def h3(x):
#     return (x[1]-x[4])
# h.append(h3)

# def g1(x):
#     return -(x[4]+10) 
# g.append(g1)

# def g2(x):
#     return -(x[0]+10) 
# g.append(g2)

# def g3(x):
#     return -(x[1]+10) 
# g.append(g3)

# def g4(x):
#     return -(x[2]+10) 
# g.append(g4)

# def g5(x):
#     return -(x[3]+10)
# g.append(g5)

# def g6(x):
#     return (x[0]-10) 
# g.append(g6)

# def g7(x):
#     return (x[1]-10)
# g.append(g7)

# def g8(x):
#     return (x[2]-10)
# g.append(g8)

# def g9(x):
#     return (x[3]-10)
# g.append(g9)

# def g10(x):
#     return (x[4]-10)
# g.append(g10)


# x0 = np.array([2.0,2.0,2.0,2.0,2.0])


# # Solution: (-33,11,27,-5,11)/43 f(x*) = 176/43

# penalty_method(f, g, h, x0)